In [1]:
import requests

url = "https://api-football-v1.p.rapidapi.com/v3/players?league=140&season=2020"

headers = {
    'x-rapidapi-host': "api-football-v1.p.rapidapi.com",
    'x-rapidapi-key': "e5cd8b6e5amsh39bfb2d72f1af22p19c842jsn8935c4a31ddb"
    }

response = requests.request("GET", url, headers=headers)

# print(response.text)

In [2]:
import json

def json2dict(str_text):
    return json.loads(str_text)

In [3]:
import urllib.parse

def build_query(base_url, endpoint, params = {}):
    
    return base_url + endpoint + ('?' if params else '') + urllib.parse.urlencode(params)

In [4]:
def call_api(base_url, endpoint, params = {}):
    
    # build url
    url = build_query(base_url, endpoint, params)
    
    # get
    headers = {
    'x-rapidapi-host': "api-football-v1.p.rapidapi.com",
    'x-rapidapi-key': "e5cd8b6e5amsh39bfb2d72f1af22p19c842jsn8935c4a31ddb"
    }

    response = requests.request("GET", url, headers=headers)
    
    return response
    
    

In [5]:
import time
import requests

def get_players_dict_data(league, season, page = 1, players_data = []):
    
    base_url = "https://api-football-v1.p.rapidapi.com/v3/"
    response = call_api(base_url, 'players', {'league':league, 'season':season, 'page':page})
    
    one_page_players_dict = json2dict(response.text)
    players_data.append(one_page_players_dict)
    
    print('League: {}, Season: {}'.format(league, season))
    print(one_page_players_dict['paging']['current'], one_page_players_dict['paging']['total'])
    
    if(one_page_players_dict['paging']['current'] < one_page_players_dict['paging']['total']):
        
        page = page + 1
        
        # time.sleep(0.1) # 根据pricing调整
        
        get_players_dict_data(league, season, page, players_data)
        
    return players_data
        
        
        
        
        
        
    

In [6]:
from pandas import json_normalize
import pandas as pd

def get_players_df_data(league, season):
    
    players_dict_data = get_players_dict_data(league, season)
    
    reponse_df = json_normalize(players_dict_data, 
                        record_path=['response'])

    statistics_df = json_normalize(players_dict_data,
                        record_path=['response', 'statistics'], meta = [['response', 'player', 'id']])

    players_df_data = pd.merge(statistics_df, reponse_df, left_on='response.player.id', right_on='player.id')

    players_df_data.drop(['response.player.id', 'statistics'], axis=1, inplace=True)
    
    players_df_data.drop(['team.logo', 'league.logo', 'player.photo'], axis=1, inplace=True)
    
    # drop duplicates & reset index
    players_df_data = players_df_data.drop_duplicates().reset_index(drop=True)
    
    return players_df_data

In [7]:
# get_players_df_data(135, 2010)

In [8]:
# players_dict_data = get_players_dict_data(251, 2010)
# players_dict_data

In [9]:
from mythread import MyThread
import pandas as pd

def get_all_players(leagues, seasons):
    
    ths = []
    for season in seasons:
        for league in leagues:
            
            th = MyThread(get_players_df_data, (league, season))
            ths.append(th)
    
    for th in ths:
        th.start()
        
    for th in ths:
        th.join()
    
    # t = pd.DataFrame()
    # for th in ths:
    #     t = pd.concat([t, th.get_result()], axis = 0)
    
    t = []
    for th in ths:
        t.append(th.get_result())
    
    rs = pd.concat(t)
    rs = rs.drop_duplicates().reset_index(drop=True)
    
    return rs

---

In [10]:
# league_ids = [140, 39, 78, 61, 135, 169, 253]
league_ids = [169]
# seasons = [year for year in range(2010, 2023)]
seasons = [2010]

In [11]:
t = get_all_players(league_ids, seasons)

League: 169, Season: 2010
1 18
League: 169, Season: 2010
2 18
League: 169, Season: 2010
3 18
League: 169, Season: 2010
4 18
League: 169, Season: 2010
5 18
League: 169, Season: 2010
6 18
League: 169, Season: 2010
7 18
League: 169, Season: 2010
8 18
League: 169, Season: 2010
9 18
League: 169, Season: 2010
10 18
League: 169, Season: 2010
11 18
League: 169, Season: 2010
12 18
League: 169, Season: 2010
13 18
League: 169, Season: 2010
14 18
League: 169, Season: 2010
15 18
League: 169, Season: 2010
16 18
League: 169, Season: 2010
17 18
League: 169, Season: 2010
18 18


In [13]:
t.columns

Index(['team.id', 'team.name', 'league.id', 'league.name', 'league.country',
       'league.flag', 'league.season', 'games.appearences', 'games.lineups',
       'games.minutes', 'games.number', 'games.position', 'games.rating',
       'games.captain', 'substitutes.in', 'substitutes.out',
       'substitutes.bench', 'shots.total', 'shots.on', 'goals.total',
       'goals.conceded', 'goals.assists', 'goals.saves', 'passes.total',
       'passes.key', 'passes.accuracy', 'tackles.total', 'tackles.blocks',
       'tackles.interceptions', 'duels.total', 'duels.won',
       'dribbles.attempts', 'dribbles.success', 'dribbles.past', 'fouls.drawn',
       'fouls.committed', 'cards.yellow', 'cards.yellowred', 'cards.red',
       'penalty.won', 'penalty.commited', 'penalty.scored', 'penalty.missed',
       'penalty.saved', 'player.id', 'player.name', 'player.firstname',
       'player.lastname', 'player.age', 'player.birth.date',
       'player.birth.place', 'player.birth.country', 'player.nationa

In [12]:
t.to_csv('league_169_year_2010_2022.csv')

---

In [29]:
import glob
import pandas as pd

file_paths = glob.glob('./*.csv')

t = []
for file_path in file_paths:
    
    df = pd.read_csv(file_path)
    
    df = df.drop(['league.flag', 'Unnamed: 0'], axis=1).drop_duplicates()
    
    t.append(df)
    
    


In [33]:
df = pd.concat(t).reset_index(drop=True)

In [34]:
df.to_csv('all_players.csv')

---

In [58]:
df = pd.read_csv('./all_players.csv')

In [59]:
df.query('`player.name` == "F. Cacciatore"')

,Unnamed: 0,team.id,team.name,league.id,league.name,league.country,league.season,games.appearences,games.lineups,games.minutes,...,player.firstname,player.lastname,player.age,player.birth.date,player.birth.place,player.birth.country,player.nationality,player.height,player.weight,player.injured
5,5,504.0,Verona,135.0,Serie A,Italy,2013,32.0,30.0,2613.0,...,Fabrizio,Cacciatore,35.0,1986-10-08,Torino,Italy,Italy,182 cm,81 kg,False
6,6,490.0,Cagliari,135.0,Serie A,Italy,2018,7.0,7.0,584.0,...,Fabrizio,Cacciatore,35.0,1986-10-08,Torino,Italy,Italy,182 cm,81 kg,False
7,7,491.0,Chievo,135.0,Serie A,Italy,2018,7.0,5.0,496.0,...,Fabrizio,Cacciatore,35.0,1986-10-08,Torino,Italy,Italy,182 cm,81 kg,False
8,8,498.0,Sampdoria,135.0,Serie A,Italy,2014,11.0,8.0,723.0,...,Fabrizio,Cacciatore,35.0,1986-10-08,Torino,Italy,Italy,182 cm,81 kg,False
9,9,491.0,Chievo,135.0,Serie A,Italy,2017,33.0,32.0,2870.0,...,Fabrizio,Cacciatore,35.0,1986-10-08,Torino,Italy,Italy,182 cm,81 kg,False
10,10,490.0,Cagliari,135.0,Serie A,Italy,2019,16.0,14.0,1155.0,...,Fabrizio,Cacciatore,35.0,1986-10-08,Torino,Italy,Italy,182 cm,81 kg,False
11,11,498.0,Sampdoria,135.0,Serie A,Italy,2010,5.0,5.0,390.0,...,Fabrizio,Cacciatore,35.0,1986-10-08,Torino,Italy,Italy,182 cm,81 kg,False
12,12,491.0,Chievo,135.0,Serie A,Italy,2016,29.0,29.0,2509.0,...,Fabrizio,Cacciatore,35.0,1986-10-08,Torino,Italy,Italy,182 cm,81 kg,False
13,13,491.0,Chievo,135.0,Serie A,Italy,2015,29.0,25.0,2309.0,...,Fabrizio,Cacciatore,35.0,1986-10-08,Torino,Italy,Italy,182 cm,81 kg,False
